# Loop 28 Analysis: Rebuild from Corners Approach

After 29 experiments, all optimization approaches converge to ~70.626. The evaluator identified a key unexplored technique:

**"Rebuild from Corners"** - Extract subsets from larger N layouts to potentially improve smaller N layouts.

This is fundamentally different from optimization - it's a CONSTRUCTIVE approach that leverages the fact that a good N=111 layout might contain a better N=50 layout than our current N=50 solution.

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
from shapely.ops import unary_union
import json

# Tree template
TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def parse_s_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

def create_tree_polygon(x, y, angle):
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def get_bounding_box_side(trees):
    all_polygons = [t for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def get_score(trees, n):
    side = get_bounding_box_side(trees)
    return (side ** 2) / n

print("Functions loaded")

Functions loaded


In [2]:
# Load baseline
df = pd.read_csv('/home/submission/submission.csv')
df['x'] = df['x'].apply(parse_s_value)
df['y'] = df['y'].apply(parse_s_value)
df['deg'] = df['deg'].apply(parse_s_value)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))

print(f"Loaded {len(df)} rows")
print(f"N values: {df['n'].min()} to {df['n'].max()}")

Loaded 20100 rows
N values: 1 to 200


In [3]:
# Calculate baseline scores for all N
baseline_scores = {}
baseline_trees = {}

for n in range(1, 201):
    group = df[df['n'] == n]
    trees = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    baseline_trees[n] = trees
    baseline_scores[n] = get_score(trees, n)

print(f"Baseline total: {sum(baseline_scores.values()):.6f}")
print(f"\nSample scores:")
for n in [1, 10, 50, 100, 150, 200]:
    print(f"  N={n}: {baseline_scores[n]:.6f}")

Baseline total: 70.625918

Sample scores:
  N=1: 0.661250
  N=10: 0.376630
  N=50: 0.360753
  N=100: 0.343397
  N=150: 0.337064
  N=200: 0.337549


In [4]:
# REBUILD FROM CORNERS APPROACH
# For each large N layout, check if subsets from corners can improve smaller N

def extract_corner_subset(trees, corner_idx, target_n):
    """
    Extract target_n trees closest to a corner of the bounding box.
    corner_idx: 0=bottom-left, 1=bottom-right, 2=top-left, 3=top-right
    """
    if len(trees) < target_n:
        return None
    
    # Get bounding box
    bounds = unary_union(trees).bounds
    corners = [
        (bounds[0], bounds[1]),  # bottom-left
        (bounds[2], bounds[1]),  # bottom-right
        (bounds[0], bounds[3]),  # top-left
        (bounds[2], bounds[3])   # top-right
    ]
    corner_x, corner_y = corners[corner_idx]
    
    # Calculate max distance from corner for each tree
    distances = []
    for i, tree in enumerate(trees):
        tree_bounds = tree.bounds
        max_dist = max(
            abs(tree_bounds[0] - corner_x),
            abs(tree_bounds[2] - corner_x),
            abs(tree_bounds[1] - corner_y),
            abs(tree_bounds[3] - corner_y)
        )
        distances.append((max_dist, i))
    
    # Sort by distance and take closest target_n trees
    distances.sort()
    selected_indices = [idx for _, idx in distances[:target_n]]
    
    return [trees[i] for i in selected_indices]

print("Corner extraction function ready")

Corner extraction function ready


In [5]:
# Test the rebuild from corners approach
# For each large N (100-200), try to extract better solutions for smaller N

improvements = []

for source_n in range(100, 201):
    source_trees = baseline_trees[source_n]
    
    for corner_idx in range(4):
        # Try to improve N values from 10 to source_n-1
        for target_n in range(10, min(source_n, 100)):
            subset = extract_corner_subset(source_trees, corner_idx, target_n)
            if subset is None:
                continue
            
            subset_score = get_score(subset, target_n)
            baseline_score = baseline_scores[target_n]
            
            if subset_score < baseline_score - 1e-9:
                improvement = baseline_score - subset_score
                improvements.append({
                    'source_n': source_n,
                    'target_n': target_n,
                    'corner': corner_idx,
                    'baseline_score': baseline_score,
                    'new_score': subset_score,
                    'improvement': improvement
                })
                print(f"IMPROVEMENT! N={target_n} from N={source_n} corner {corner_idx}: {baseline_score:.6f} -> {subset_score:.6f} ({improvement:.6f})")

print(f"\nTotal improvements found: {len(improvements)}")
if improvements:
    total_improvement = sum(imp['improvement'] for imp in improvements)
    print(f"Total potential improvement: {total_improvement:.6f}")


Total improvements found: 0


In [6]:
# If no improvements from large N, try the reverse:
# Check if smaller N layouts can be combined to improve larger N
# This is less likely but worth checking

print("\nChecking if any N values have suboptimal solutions...")
print("\nAnalyzing efficiency by N range:")

for start_n in [1, 11, 51, 101, 151]:
    end_n = min(start_n + 49, 200)
    range_scores = [baseline_scores[n] for n in range(start_n, end_n + 1)]
    avg_score = np.mean(range_scores)
    print(f"  N={start_n}-{end_n}: avg score = {avg_score:.6f}")


Checking if any N values have suboptimal solutions...

Analyzing efficiency by N range:
  N=1-50: avg score = 0.380666
  N=11-60: avg score = 0.365818
  N=51-100: avg score = 0.352268
  N=101-150: avg score = 0.342725
  N=151-200: avg score = 0.336859


In [7]:
# Check the theoretical lower bound
# For a single tree, the minimum bounding square has side = sqrt(area of tree)
# Tree area can be calculated from the polygon

tree_poly = Polygon(TREE_TEMPLATE)
tree_area = tree_poly.area
print(f"Single tree area: {tree_area:.6f}")

# For N trees, theoretical minimum is when trees pack perfectly
# Score = S^2/N, where S^2 >= N * tree_area (if perfect packing)
# So minimum score per N >= tree_area = {tree_area:.6f}

print(f"\nTheoretical minimum score per N (perfect packing): {tree_area:.6f}")
print(f"\nActual scores vs theoretical:")
for n in [1, 10, 50, 100, 200]:
    actual = baseline_scores[n]
    efficiency = tree_area / actual * 100
    print(f"  N={n}: actual={actual:.6f}, efficiency={efficiency:.1f}%")

Single tree area: 0.245625

Theoretical minimum score per N (perfect packing): 0.245625

Actual scores vs theoretical:
  N=1: actual=0.661250, efficiency=37.1%
  N=10: actual=0.376630, efficiency=65.2%
  N=50: actual=0.360753, efficiency=68.1%
  N=100: actual=0.343397, efficiency=71.5%
  N=200: actual=0.337549, efficiency=72.8%


In [8]:
# The key insight: We need to find WHERE the inefficiency is
# Let's look at the score contribution by N range

print("Score contribution by N range:")
ranges = [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]

for start, end in ranges:
    range_total = sum(baseline_scores[n] for n in range(start, end + 1))
    range_count = end - start + 1
    print(f"  N={start}-{end}: {range_total:.4f} ({range_count} values, avg={range_total/range_count:.4f})")

print(f"\nTotal: {sum(baseline_scores.values()):.6f}")
print(f"Target: 68.919154")
print(f"Gap: {sum(baseline_scores.values()) - 68.919154:.6f}")

Score contribution by N range:
  N=1-10: 4.3291 (10 values, avg=0.4329)
  N=11-50: 14.7042 (40 values, avg=0.3676)
  N=51-100: 17.6134 (50 values, avg=0.3523)
  N=101-150: 17.1363 (50 values, avg=0.3427)
  N=151-200: 16.8430 (50 values, avg=0.3369)

Total: 70.625918
Target: 68.919154
Gap: 1.706764


In [9]:
# Let's identify the N values with the WORST efficiency
# These are the ones where improvement is most likely

efficiencies = []
for n in range(1, 201):
    actual = baseline_scores[n]
    efficiency = tree_area / actual * 100
    efficiencies.append((n, actual, efficiency))

# Sort by efficiency (lowest first = worst)
efficiencies.sort(key=lambda x: x[2])

print("TOP 20 WORST EFFICIENCY N VALUES:")
print("(These are the best candidates for improvement)")
for n, score, eff in efficiencies[:20]:
    print(f"  N={n}: score={score:.6f}, efficiency={eff:.1f}%")

TOP 20 WORST EFFICIENCY N VALUES:
(These are the best candidates for improvement)
  N=1: score=0.661250, efficiency=37.1%
  N=2: score=0.450779, efficiency=54.5%
  N=3: score=0.434745, efficiency=56.5%
  N=5: score=0.416850, efficiency=58.9%
  N=4: score=0.416545, efficiency=59.0%
  N=7: score=0.399897, efficiency=61.4%
  N=6: score=0.399610, efficiency=61.5%
  N=9: score=0.387415, efficiency=63.4%
  N=8: score=0.385407, efficiency=63.7%
  N=15: score=0.376950, efficiency=65.2%
  N=10: score=0.376630, efficiency=65.2%
  N=21: score=0.376451, efficiency=65.2%
  N=20: score=0.376057, efficiency=65.3%
  N=22: score=0.375258, efficiency=65.5%
  N=11: score=0.374924, efficiency=65.5%
  N=16: score=0.374128, efficiency=65.7%
  N=26: score=0.373997, efficiency=65.7%
  N=12: score=0.372724, efficiency=65.9%
  N=13: score=0.372294, efficiency=66.0%
  N=25: score=0.372144, efficiency=66.0%


In [10]:
# Summary and next steps
print("="*70)
print("LOOP 28 ANALYSIS SUMMARY")
print("="*70)

print(f"\nCurrent best: {sum(baseline_scores.values()):.6f}")
print(f"Target: 68.919154")
print(f"Gap: {sum(baseline_scores.values()) - 68.919154:.6f} ({(sum(baseline_scores.values()) - 68.919154) / 68.919154 * 100:.2f}%)")

print(f"\nRebuild from corners found {len(improvements)} improvements")

print("\nKEY INSIGHTS:")
print("1. The baseline is at a VERY STRONG local optimum")
print("2. All optimization approaches (SA, bbox3, etc.) converge to same solution")
print("3. The 'rebuild from corners' approach found NO improvements")
print("4. This suggests the baseline is near-optimal for each N")

print("\nPOSSIBLE PATHS FORWARD:")
print("1. Find better solutions from external sources (other Kaggle submissions)")
print("2. Manual editing using the Interactive Editor")
print("3. Domain-specific techniques (k-mer exploration, efficient basin search)")
print("4. Accept that the target may require techniques not publicly available")

LOOP 28 ANALYSIS SUMMARY

Current best: 70.625918
Target: 68.919154
Gap: 1.706764 (2.48%)

Rebuild from corners found 0 improvements

KEY INSIGHTS:
1. The baseline is at a VERY STRONG local optimum
2. All optimization approaches (SA, bbox3, etc.) converge to same solution
3. The 'rebuild from corners' approach found NO improvements
4. This suggests the baseline is near-optimal for each N

POSSIBLE PATHS FORWARD:
1. Find better solutions from external sources (other Kaggle submissions)
2. Manual editing using the Interactive Editor
3. Domain-specific techniques (k-mer exploration, efficient basin search)
4. Accept that the target may require techniques not publicly available
